![Renode](https://dl.antmicro.com/projects/renode/renode.png)
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/antmicro/tensorflow-zephyr-vexriscv-examples/blob/main/examples/magic-wand/magic_wand_build_binary.ipynb"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-zephyr-vexriscv-examples/main/examples/.static/view-in-colab.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/antmicro/tensorflow-zephyr-vexriscv-examples/blob/main/examples/magic-wand/magic_wand_build_binary.ipynb"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-zephyr-vexriscv-examples/main/examples/.static/view-ipynb.png" />View ipynb on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/antmicro/tensorflow-zephyr-vexriscv-examples/blob/main/examples/magic-wand/magic_wand.py"><img src="https://raw.githubusercontent.com/antmicro/tensorflow-zephyr-vexriscv-examples/main/examples/.static/view-source.png" />View Python source on GitHub</a>
  </td>
</table>

## Install requirements

In [ ]:
!pip install -q git+https://github.com/antmicro/pyrenode.git git+https://github.com/antmicro/renode-colab-tools.git
!mkdir -p renode && cd renode && wget https://dl.antmicro.com/projects/renode/builds/renode-latest.linux-portable.tar.gz && tar -xzf renode-latest.linux-portable.tar.gz --strip 1
!pip install -q -r renode/tests/requirements.txt
!git clone --quiet --recurse-submodules https://github.com/antmicro/tensorflow-zephyr-vexriscv-examples.git

import os
from renode_colab_tools import metrics
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']
os.environ['TENSORFLOW_PATH'] = os.getcwd()+"/tensorflow-zephyr-vexriscv-examples/tensorflow"
os.environ['ZEPHYR_SDK_INSTALL_DIR'] = "/opt/zephyr-sdk"
os.environ['ZEPHYR_TOOLCHAIN_VARIANT'] = "zephyr"

In [ ]:
!sudo apt update && sudo apt install -y cmake ninja-build gperf ccache dfu-util device-tree-compiler wget python python3-pip python3-setuptools python3-tk python3-wheel xz-utils file make gcc gcc-multilib locales tar curl unzip xxd
!pip install --upgrade cmake virtualenv pillow
!wget https://github.com/zephyrproject-rtos/sdk-ng/releases/download/v0.11.2/zephyr-sdk-0.11.2-setup.run
!chmod +x zephyr-sdk-0.11.2-setup.run
!./zephyr-sdk-0.11.2-setup.run -- -y -d $ZEPHYR_SDK_INSTALL_DIR
!sudo apt install patchelf -y
!sudo patchelf --set-interpreter /lib64/ld-linux-x86-64.so.2 $ZEPHYR_SDK_INSTALL_DIR/sysroots/x86_64-pokysdk-linux/usr/bin/dtc
!cd $TENSORFLOW_PATH && make -f tensorflow/lite/micro/tools/make/Makefile TARGET=zephyr_vexriscv magic_wand_bin
!mkdir -p binaries/magic_wand && cp $TENSORFLOW_PATH/tensorflow/lite/micro/tools/make/gen/zephyr_vexriscv_x86_64_default/magic_wand/build/zephyr/zephyr.elf binaries/magic_wand/magic_wand_zephyr.elf

## Run a magic-wand example in Renode

In [ ]:
import time
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)
tell_renode('using sysbus')
tell_renode('mach create')
tell_renode('machine LoadPlatformDescription @tensorflow-zephyr-vexriscv-examples/examples/litex-vexriscv-tflite.repl')
tell_renode('sysbus LoadELF @binaries/magic_wand/magic_wand_zephyr.elf')

tell_renode('uart CreateFileBackend @uart.dump true')
tell_renode('logLevel 3')
tell_renode('machine EnableProfiler "metrics.dump"')
tell_renode('i2c.adxl345 MaxFifoDepth 1')
tell_renode('i2c.adxl345 FeedSample @tensorflow-zephyr-vexriscv-examples/examples/magic-wand/angle.data')
tell_renode('s')
time.sleep(5) #waits for creating uart.dump
!timeout 60 tail -c+2 -f renode/uart.dump | sed '/\* \* \* \* \* \* \* \*/ q'
tell_renode('q')
expect_cli('Renode is quitting')
time.sleep(1) #wait not to kill Renode forcefully
shutdown_renode()

## Renode metrics analysis

In [ ]:
from renode.tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('renode/metrics.dump')

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_executed_instructions(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_memory_access(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_exceptions(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_peripheral_access(parser)